<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

In [2]:
%conda init
%conda activate aq

no change     C:\Users\patri\miniconda3\Scripts\conda.exe
no change     C:\Users\patri\miniconda3\Scripts\conda-env.exe
no change     C:\Users\patri\miniconda3\Scripts\conda-script.py
no change     C:\Users\patri\miniconda3\Scripts\conda-env-script.py
no change     C:\Users\patri\miniconda3\condabin\conda.bat
no change     C:\Users\patri\miniconda3\Library\bin\conda.bat
no change     C:\Users\patri\miniconda3\condabin\_conda_activate.bat
no change     C:\Users\patri\miniconda3\condabin\rename_tmp.bat
no change     C:\Users\patri\miniconda3\condabin\conda_auto_activate.bat
no change     C:\Users\patri\miniconda3\condabin\conda_hook.bat
no change     C:\Users\patri\miniconda3\Scripts\activate.bat
no change     C:\Users\patri\miniconda3\condabin\activate.bat
no change     C:\Users\patri\miniconda3\condabin\deactivate.bat
no change     C:\Users\patri\miniconda3\Scripts\activate
no change     C:\Users\patri\miniconda3\Scripts\deactivate
no change     C:\Users\patri\miniconda3\etc\profile.d\


CondaError: Run 'conda init' before 'conda activate'



### <span style='color:#ff5f27'> 📝 Imports

In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [7]:
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value


In [8]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login()
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2024-12-16 22:37:32,579 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-16 22:37:32,584 INFO: Initializing external client
2024-12-16 22:37:32,585 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-16 22:37:34,569 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1100948


'{"country": "Sweden", "city": "Stockholm", "street": "Hornsgatan 108", "aqicn_url": "https://api.waqi.info/feed/@10009", "latitude": "59.3173", "longitude": "18.0489"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [9]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [10]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,5.0,Sweden,Stockholm,Hornsgatan 108,2024-12-16,https://api.waqi.info/feed/@10009


In [11]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [12]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 43.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-12-17,1.50,0.0,21.194187,290.897827,Stockholm
1,2024-12-18,2.00,1.1,19.486609,154.855255,Stockholm
2,2024-12-19,4.65,0.0,18.598450,227.353333,Stockholm
3,2024-12-20,0.00,0.0,17.786331,305.942200,Stockholm
4,2024-12-21,0.95,0.0,10.823973,266.186005,Stockholm
5,2024-12-22,2.60,0.0,10.315115,209.248734,Stockholm
6,2024-12-23,0.90,0.4,14.081478,355.601379,Stockholm
7,2024-12-24,0.95,0.6,14.565548,230.013168,Stockholm
8,2024-12-25,-0.50,0.0,8.654986,286.927582,Stockholm
9,2024-12-26,3.05,0.0,16.375053,303.340729,Stockholm


In [13]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [14]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-12-16 22:38:17,930 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1100948/fs/1091651/fg/1390951


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:02 | Remaining Time: 00:00


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 693249
         }
       },
       "result": {
         "observed_value": 5.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-17T01:38:17.000930Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [15]:
# Insert new data
weather_fg.insert(daily_df)

2024-12-16 22:38:27,452 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1100948/fs/1091651/fg/1390952


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 693251
         }
       },
       "result": {
         "observed_value": 8.654986381530762,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-17T01:38:27.000451Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
